In [1]:
import pandas as pd
#user_books_rate_raw_df = pd.read_json(open("../../../run_snapshot_26Dec21_1549/user_book_rates_array.json"), orient="records")
#user_books_rate_raw_df = pd.read_json(open("../../../run_snapshot_26Dec21_1549/user_book_rates_array.json"), orient="records")
user_books_rate_raw_df = pd.read_json(open("../../../run_snapshots/run_snapshot_29Dec21_1248/user_book_rates_array.json"), orient="records")
user_books_rate_raw_df

,userId,book_url,rate
0,noorhe,/book/1006273495-poslednij-astronavt-devid-vel...,4.0
1,noorhe,/book/1006273495-poslednij-astronavt-devid-vel...,4.0
2,noorhe,/book/1000202350-letopisi-belgariada-poslednya...,5.0
3,noorhe,/book/1001893850-castle-of-wizardry-david-eddings,5.0
4,noorhe,/book/1001997048-magicians-gambit-david-eddings,5.0
...,...,...,...
130234,TarkowskiReheating,/book/1000438866-zapiski-kota-shashlyka-kompyu...,0.0
130235,TarkowskiReheating,/book/1000242596-zapiski-nevesty-programmista-...,0.0
130236,TarkowskiReheating,/book/1006129594-igrat-chtoby-zhit-kniga-9-dor...,0.0
130237,TarkowskiReheating,/book/1005436720-igrat-chtoby-zhit-kniga-8-put...,0.0


In [3]:
operations_per_user = pd.read_csv(open("../../../run_snapshots/run_snapshot_29Dec21_1248/read_list_operations_per_user.csv"))
only_push = operations_per_user[operations_per_user.op == "push"]
only_pull = operations_per_user[operations_per_user.op == "pull"]
only_push_grouped_by_user = only_push.groupby(by="user").count()
only_pull_grouped_by_user = only_pull.groupby(by="user").count()
user_ops_count_merged = only_push_grouped_by_user.merge(only_pull_grouped_by_user, on="user")
users_with_completed_ops = user_ops_count_merged[user_ops_count_merged.op_y == user_ops_count_merged.op_x]
users_with_completed_ops_more_than_one = users_with_completed_ops[users_with_completed_ops.op_y > 1]
print(len(users_with_completed_ops))
users_with_completed_ops

450


,op_x,op_y
user,,
2762194,1,1
6even,3,3
AVESON_BOOK,16,16
Adivo,49,49
AirGard,1,1
...,...,...
yuliadnb,4,4
yulyavasileva90,1,1
zairahuseinova26,3,3


In [4]:
user_books_rate_completed_raw_df = user_books_rate_raw_df.merge(users_with_completed_ops, left_on="userId", right_on="user")
user_books_rate_completed_raw_df

,userId,book_url,rate,op_x,op_y
0,noorhe,/book/1006273495-poslednij-astronavt-devid-vel...,4.0,13,13
1,noorhe,/book/1006273495-poslednij-astronavt-devid-vel...,4.0,13,13
2,noorhe,/book/1000202350-letopisi-belgariada-poslednya...,5.0,13,13
3,noorhe,/book/1001893850-castle-of-wizardry-david-eddings,5.0,13,13
4,noorhe,/book/1001997048-magicians-gambit-david-eddings,5.0,13,13
...,...,...,...,...,...
118769,FossaWhippets,/book/1002131181-knizhnyj-vor-markus-zusak,0.0,5,5
118770,FossaWhippets,/book/1002006306-451-po-farengejtu-rej-bredberi,0.0,5,5
118771,FossaWhippets,/book/1002888124-sem-smertej-eveliny-hardkasl-...,4.0,5,5
118772,FossaWhippets,/book/1002903202-book-love-debbie-tung,0.0,5,5


Lets try to analyze user rate using collaborative approach based on cosine distanse between user book rate lists.
Now in dataset each json represents one rate from a user to a particular book.
We need to transform the data to make each row represent all rate from a user (row) to books (cols)

In [8]:
grouped_by_userid = user_books_rate_completed_raw_df.groupby("userId")

In [6]:
#grouped_by_userid_sample = user_books_rate_raw_sample_df.groupby("userId")
print(len(grouped_by_userid.groups))
print(grouped_by_userid["book_url"].count())
count_books_by_userid = grouped_by_userid["book_url"].count()
user_book_rates_grouped_counted = user_books_rate_completed_raw_df.merge(count_books_by_userid, on="userId")
user_book_rates_grouped_counted = user_book_rates_grouped_counted[user_book_rates_grouped_counted["book_url_y"] > 50]
user_book_rates_grouped_counted_grpby_userid = user_book_rates_grouped_counted.groupby("userId")
user_book_rates_grouped_counted_grpby_userid
print(len(user_book_rates_grouped_counted_grpby_userid.groups))

426
userId
2762194               7
6even                41
AVESON_BOOK         311
Adivo               972
AirGard              13
                   ... 
yukio0n               8
yuliadnb             80
zairahuseinova26     50
zkyprod              43
zoudi               419
Name: book_url, Length: 426, dtype: int64
317


In [17]:
from scipy import spatial

columns = ["user_left", "user_right", "set_similarity", "rating_similarity", "total_similarity"]
result = pd.DataFrame([], columns = columns)
for user_name1 in grouped_by_userid.groups:
    user_rate_list1 = grouped_by_userid.get_group(user_name1)
    user_rate_list1_no_na_rates = user_rate_list1.dropna(subset=["rate"])
    user_rate_list1_no_na_rates_len = len(user_rate_list1_no_na_rates)
    for user_name2 in grouped_by_userid.groups:
        if (user_name1 is not user_name2):
            user_rate_list2 = grouped_by_userid.get_group(user_name2)
            #df.loc[df['column_name'] == some_value]
            user_rate_list2_no_na_rates = user_rate_list2.dropna(subset=["rate"])
            user_rate_list2_no_na_rates_len = len(user_rate_list2_no_na_rates)
            (more, less) = (user_rate_list1_no_na_rates_len, user_rate_list2_no_na_rates_len) if user_rate_list1_no_na_rates_len > user_rate_list2_no_na_rates_len else (user_rate_list2_no_na_rates_len, user_rate_list1_no_na_rates_len)
            koef = less / more
            if(koef <= 0.7):
                continue
            user_rate_list1_no_zero_rates = user_rate_list1_no_na_rates.loc[user_rate_list1_no_na_rates['rate'] != 0]
            user_rate_list2_no_zero_rates = user_rate_list2_no_na_rates.loc[user_rate_list2_no_na_rates['rate'] != 0]
            joined_read_lists = user_rate_list1_no_zero_rates.merge(user_rate_list2_no_zero_rates, on="book_url")
            if(not joined_read_lists.empty):
                user1_books = set(user_rate_list1.book_url)
                user2_books = set(user_rate_list2.book_url)
                join_size = len(user1_books) + len(user2_books)
                intersection_size = len(user1_books.intersection(user2_books))
                by_set_similarity = intersection_size / join_size
                left_rates = joined_read_lists.rate_x.to_numpy()
                right_rates = joined_read_lists.rate_y.to_numpy()
                cosine_distance = spatial.distance.cosine(left_rates, right_rates)
                total_similarity = by_set_similarity*cosine_distance
                df = pd.DataFrame([(user_name1, user_name2, by_set_similarity, cosine_distance, total_similarity)], columns = columns)
                if(total_similarity != 0):
                    result = result.append(df)
print(result)

      user_left        user_right  set_similarity  rating_similarity  \
0   AVESON_BOOK     AnatolyEmanov        0.019856           0.018735   
0   AVESON_BOOK       AnyaDremina        0.017974           0.012824   
0   AVESON_BOOK          Bonifika        0.031193           0.020141   
0   AVESON_BOOK  BrummittGlidings        0.023529           0.016356   
0   AVESON_BOOK          Celin136        0.006221           0.061897   
..          ...               ...             ...                ...   
0         zoudi       marinabooks        0.005025           0.000488   
0         zoudi    nataliaperna18        0.006443           0.027113   
0         zoudi            svelby        0.004396           0.007698   
0         zoudi        tbukalaeva        0.002706           0.003759   
0         zoudi        umka_pumka        0.003480           0.013512   

    total_similarity  
0           0.000372  
0           0.000230  
0           0.000628  
0           0.000385  
0           0.000385

In [19]:
result[result.user_left == "zoudi"]

,user_left,user_right,set_similarity,rating_similarity,total_similarity
0,zoudi,AnyaDremina,0.005556,0.138187,0.000768
0,zoudi,Belera,0.005388,0.006116,0.000033
0,zoudi,BoniferReservable,0.002339,0.029857,0.000070
0,zoudi,Bookngriller,0.008511,0.052322,0.000445
0,zoudi,Celin136,0.002663,0.058258,0.000155
0,zoudi,DolingerZythumla,0.008838,0.003727,0.000033
0,zoudi,ElenaLatysheva,0.003333,0.020204,0.000067
0,zoudi,EliaStrange,0.004104,0.005063,0.000021
0,zoudi,Ellina,0.007042,0.032978,0.000232
0,zoudi,Gold_Sister,0.005020,0.081661,0.000410


In [20]:
result.to_csv("../../../run_snapshots/run_snapshot_29Dec21_1248/recommendations_result.csv")

In [22]:
result_max = result[result.total_similarity == result.total_similarity.max()]
result_max

,user_left,user_right,set_similarity,rating_similarity,total_similarity
0,KotinNa,shmat88,0.028986,0.324912,0.009418
0,shmat88,KotinNa,0.028986,0.324912,0.009418


In [ ]:
result

Let's check, how many and which users' read lists is been completed